# Fig6

In [1]:
from zebrafish_ms2_paper.gillespie_simulations_delay import Params, simulate_multiple_copies, sim_ms2, hill_function
from zebrafish_ms2_paper.trace_analysis import binarize_trace, get_on_and_off_times, get_burst_durations, get_burst_inactive_durations, extract_traces
from zebrafish_ms2_paper.utils import pboc_rc, style_axes, colors, fontsize
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import numpy as np
from multiprocessing import Pool
import pandas as pd
import pickle

In [2]:
rcParams.update(pboc_rc)
rcParams['pdf.fonttype'] = 42

In [3]:
colors

{'green': '#7AA974',
 'light_green': '#BFD598',
 'pale_green': '#DCECCB',
 'yellow': '#EAC264',
 'light_yellow': '#F3DAA9',
 'pale_yellow': '#FFEDCE',
 'blue': '#738FC1',
 'light_blue': '#A9BFE3',
 'pale_blue': '#C9D7EE',
 'red': '#D56C55',
 'light_red': '#E8B19D',
 'pale_red': '#F1D4C9',
 'purple': '#AB85AC',
 'light_purple': '#D4C2D9',
 'dark_green': '#7E9D90',
 'dark_brown': '#905426'}

In [4]:
fontsize = 9
linewidth = 2
run_sim = True
n_replicates = 15
markersize = 8
#bins = np.arange(3, 55)
bins = np.linspace(3, 55, 12)
#bins = np.linspace(3, 40, 15)


In [5]:
%matplotlib qt

In [6]:
"""specify the path to the folder of Supplemental Data Files"""
# REPLACE THIS WITH YOUR PATH
datadir = r'/home/brandon/Downloads/SupplementalDataFiles'

# define paths to particular files
path_to_spots = datadir + '/Supplemental_Data_File_2-Spots_Curated.pkl'
path_to_non_blank_timepoints = datadir + '/Supplemental_Data_File_3-Non-Blank_Time_Points.pkl'


In [7]:
df = pd.read_pickle(path_to_spots)

In [8]:
intens = df.gauss3d_dog.values
brightest_spot_intensity = np.quantile(intens[intens > 0], q=(0.95,))
brightest_spot_intensity / 10 / 0.29 / 4

array([4575.5862069])

## Fig6B,C,D
Plot example traces for the three burst regulation models. Here we also plot the predicted protein traces, though we removed these from the final figure for simplicity. This code generates new stochastic simulations, so the traces won't be exactly the same was what is plotted in the figure.

In [9]:
plt.close('all')

In [10]:
f, axs = plt.subplots(2,4)

In [11]:
def plot_example_trace(p, f=None, axs=None, burn_in_time=45):
    if f is None:
        f, axs = plt.subplots(2, 1, figsize=(4,3))

    X, tvec, p = simulate_multiple_copies(p)
    state = X[:, 0]
    mrna = X[:, -2]
    protein = X[:, -1]
    
    start_id = np.where(tvec > burn_in_time)[0][0]
    state = state[start_id:]
    mrna = mrna[start_id:]
    protein = protein[start_id:]
    tvec = tvec[start_id:] - burn_in_time
    X = X[start_id:]

    production_rate = p.transcription_rate_0 + p.transcription_rate_1 * hill_function(protein, p.KD_transcription_rate, p.n)
    #detection_threshold = 0.0
    relevant_production_rate = production_rate[np.where(tvec > burn_in_time)[0]]
    m = np.mean(np.log10(relevant_production_rate[relevant_production_rate > 0]))
    s = np.std(np.log10(relevant_production_rate[relevant_production_rate > 0]))
    detection_threshold = 0.22 * 10 ** (m + s) * w
    #detection_threshold = 0.2 * np.max(production_rate[np.where(tvec > burn_in_time)[0][0]]) * w

    ms2, t_obs = sim_ms2(state, tvec, production_rate, w, delta_t, sigma, detection_threshold)

    inferred_state = binarize_trace(ms2, t_obs, thresh=1e-1, window_size=3)

    # plot
    # state
    ax = axs[0]
    ax.plot(tvec, state, 'k-', linewidth=linewidth)

    #plt.title('amplitude regulation', fontsize=fontsize)
    #ax.set_ylabel('promoter \nstate \n(0, 1)', fontsize=fontsize)
    ax.set_ylim([0,1.2])
    ax.set_xticks([0, 60, 120, 180], labels=[])
    ax.set_yticks([0, 1], labels=[])
    ax = style_axes(ax, fontsize=fontsize)

    # MS2
    ax = axs[1]
    ax.plot(tvec, protein / np.max(protein), '-', linewidth=linewidth, color=colors['purple'], label='protein')
    ax.plot(t_obs, ms2 / np.max(ms2), '-', linewidth=linewidth, color=colors['green'], label='MS2')
    #ax.plot(t_obs, inferred_state, '-', linewidth=linewidth, color='k', label='inferred state')
    #ax.plot(tvec, production_rate / np.max(production_rate), '-', linewidth=linewidth, color=colors['blue'], label='transcription rate')
    
    #plt.title('amplitude regulation', fontsize=fontsize)
    #ax.set_ylabel('simulated \nher1 activity \n(fraction of max)', fontsize=fontsize)
    ax.set_xlabel('time (min)', fontsize=fontsize)
    ax.set_ylim([0,1.5])
    ax.set_xticks([0, 60, 120, 180])
    ax.set_yticks([0, 1], labels=[])
    #ax.legend(fontsize=fontsize, facecolor='none', loc=1, ncol=2)
    ax = style_axes(ax, fontsize=fontsize)

    #f.tight_layout()
    
    return f, axs, X, tvec

In [12]:
"""fixed model params"""
elongation_time = 0.29
delta_t = 1.0
w = elongation_time / delta_t
sigma = 0.2

In [13]:
"""amplitude regulation, no bursts"""
p = Params()
p.initial_state = np.array([1])
p.Tmax = 240
p.k_off0 = 0#0.3#0.025
p.k_off1 = 0.0#0.0
p.k_on0 = 1000#0.1#.0
p.k_on1 = 0#2.5
p.transcription_rate_0 = 0

p.translation_rate = 4.5
p.transcription_rate_1 = 10.0 / len(p.initial_state)

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.delay = 7.5

p.KD_transcription_rate = 100
p.n = 3

_, _, X, tvec = plot_example_trace(p, f=f, axs=axs[:,0])

In [14]:
"""k_on regulation"""
p = Params()
p.initial_state = np.array([1])#, 1, 1, 1])
p.Tmax = 240#1000
p.k_off0 = 0.2#0.2#2.2#0.025
p.k_off1 = 0.0#0.0
p.k_on0 = 0.0#0.1#2.2
p.k_on1 = 3#2.5
p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

p.translation_rate = 4.5#4.5# max possible 16
p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.delay = 0.0

p.KD_k_on = 10
p.n = 10

_, _, X, tvec = plot_example_trace(p, f=f, axs=axs[:,1])

In [15]:
"""k_off regulation"""
p = Params()
p.initial_state = np.array([1])#, 1, 1, 1])
p.Tmax = 240#1000
p.k_off0 = 0.0#0.2#2.2#0.025
p.k_off1 = 1000*0.08#0.0
p.k_on0 = 0.05#0.1#2.2
p.k_on1 = 0.0#2.5
p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

p.translation_rate = 4.5#4.5# max possible 16
p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23
p.delay = 0

p.KD_k_on = 10
p.KD_k_off = 800
p.n = 20
_, _, X, tvec = plot_example_trace(p, f=f, axs=axs[:,2])

In [16]:
"""k_on and k_off regulation, complete regulation"""
p = Params()
p.number_of_random_numbers_to_pregenerate = 1e6
p.initial_state = np.array([1])#, 1, 1, 1])
p.Tmax = 240#1000
p.k_off0 = 0.0#0.2#0.2#2.2#0.025
p.k_off1 = 0.2#0.2#0.0
p.k_on0 = 0.0#0.1#0.1#2.2
p.k_on1 = 0.2#0.1#2.5
p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

p.translation_rate = 4.5#4.5# max possible 16
p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23
p.delay = 0

p.KD_k_on = 10
p.KD_k_off = 100
p.n = 10

_, _, X, tvec = plot_example_trace(p, f=f, axs=axs[:,3])

In [17]:
axs[0, 0].set_yticks([0, 1], labels=[0, 1])
axs[0, 0].set_ylabel('promoter \nstate \n(0, 1)', fontsize=fontsize)

axs[1, 0].set_yticks([0, 1], labels=[0, 1])
axs[1, 0].set_ylabel('simulated \nher1 activity \n(fraction of max)', fontsize=fontsize)
axs[1, 0].legend(fontsize=fontsize, facecolor='none', loc=1, ncol=2)

In [47]:
plt.savefig(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/figures/example_simluated_traces.pdf')